# Imports

In [15]:
# IMPORT
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.path import Path
import numpy as np
import seaborn as sns

# bokeh
import bokeh.palettes
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, Legend, ColumnDataSource, Title
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.models import GeoJSONDataSource


output_notebook()
np.random.seed(42)

Loading BokehJS ...

# Constants

In [ ]:
district_colors = sns.color_palette('colorblind', 21)


# Madrid Central

We are going to analyze the impact of Madrid Central both from an air quality and a traffic viewpoint.

## Air Quality

In [5]:
# load air quality stations
df_stations = pd.read_csv('shared_data/air_quality/air_quality_stations.csv')

# load magnitud table
df_magnitud = pd.read_csv('shared_data/air_quality/air_quality_magnitud.csv', sep=';')

# load air quality data
df = pd.read_csv('data/air_quality_data.csv')

# converting Date to datetime type
df["datetime"] = pd.to_datetime(df["datetime"])

# merge with air quality stations
df = pd.merge(df, df_stations, left_on = 'PUNTO_MUESTREO', right_on='punto_muestreo', how='left').drop('PUNTO_MUESTREO', axis=1)

# merge with air quality magnitud
df = pd.merge(df, df_magnitud, left_on = 'MAGNITUD', right_on='magnitud_id', how='left').drop('MAGNITUD', axis=1)

df.head()

,PROVINCIA,MUNICIPIO,ESTACION,value,datetime,punto_muestreo,name,longitude,latitude,altitude,utm_x,utm_y,magnitud_id,formula,unit_per_m3
0,28,79,4,7.0,2016-04-01 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg
1,28,79,4,8.0,2016-04-02 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg
2,28,79,4,10.0,2016-04-03 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg
3,28,79,4,7.0,2016-04-04 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg
4,28,79,4,8.0,2016-04-05 01:00:00,28079004,Plaza de España,-3.712197,40.423883,637,-413239.904502,4.927732e+06,1,SO2,µg


In [8]:
print('AIR QUALITY STATION NAMES:')
print([name for name in df.name.unique()])

AIR QUALITY STATION NAMES:
['Plaza de España', 'Escuelas Aguirre', 'Ramón y Cajal', 'Arturo Soria', 'Villaverde', 'Farolillo', 'Casa de Campo', 'Barajas Pueblo', 'Plaza del Carmen', 'Moratalaz', 'Cuatro Caminos', 'Barrio del Pilar', 'Vallecas', 'Méndez Álvaro', 'Castellana', 'Retiro', 'Plaza Castilla', 'Ensanche de Vallecas', 'Urbanización Embajada', 'Plaza Elíptica', 'Sanchinarro', 'El Pardo', 'Juan Carlos I', 'Tres Olivos']


In [14]:
# MAP

# load MC area
cm_points = pd.read_csv('shared_data/districts/central_madrid_points.csv')

points = df_stations[["utm_x", "utm_y"]].values
path = Path(cm_points[["utm_x", "utm_y"]].values)
points_in_path_mask = path.contains_points(points)

# plot map
p = figure(title="Air quality stations in Madrid", x_axis_type="mercator", y_axis_type="mercator")

source = ColumnDataSource(df_stations)
cr = p.circle(x="utm_x", y="utm_y",  size=10, source=source)

cartodb = get_provider(CARTODBPOSITRON)
p.add_tile(cartodb)

p.add_tools(HoverTool(tooltips=[('Name', '@name')], renderers=[cr]))

show(p)